In [ ]:
from mpcontribs.client import load_client
from mpcontribs.io.core.recdict import RecursiveDict
from mpcontribs.io.core.components.hdata import HierarchicalData
from mpcontribs.io.core.components.tdata import Table
from mpcontribs.io.core.components.gdata import Plot

In [ ]:
api_key = '<your-api-key-here>' # from https://portal.mpcontribs.org/dashboard
client = load_client(api_key)

## project info & overview table

In [ ]:
project = 'carrier_transport'
info = client.projects.get_entry(project=project).response().result
RecursiveDict(info)

In [ ]:
data = client.projects.get_table(project=project).response().result
Table(data['items'], project=project, api_key=api_key)

## specific contribution

In [ ]:
cid = '5ac0a333d4f1443de26744ab'
contrib = client.contributions.get_entry(cid=cid).response().result

### hierarchical data

In [ ]:
HierarchicalData(contrib['content']['data']) # dictionary

### tabular data

In [ ]:
tid = contrib['content']['tables'][0]
table = client.tables.get_entry(tid=tid).response().result
Table.from_dict(table) # Pandas DataFrame format

In [ ]:
Plot.from_dict(table) # plotly graph

## query

In [ ]:
# see https://portal.mpcontribs.org/<project> and
# https://api.mpcontribs.org/#/contributions/get_entries

per_page = 20 # 20 is max retrievable contributions at once
fields = ['<S>', '<σ>', '<S²σ>'] # which data fields to retrieve
mask = [f'content.data.{field}' for field in fields]
mask += ['content.data.formula', 'identifier']
filters = [
    'formula__contains:Li3',
    '<σ>.p__lt:2e15', '<σ>.n__lt:2e15',
] # implicitly AND, only last filter kept for duplicate field/operator combinations

contribs = client.contributions.get_entries(
    projects=[project], filters=filters, mask=mask, per_page=per_page
).response().result # -> see pagination

print('found', len(contribs), 'materials')
contribs[0]

### pagination

In [ ]:
# only run this once you've optimized masks and filters to what you actually need (see above)
all_contribs = []
while 1:
    page = int(len(all_contribs) / per_page) + 1
    contribs = client.contributions.get_entries(
        projects=[project], filters=filters, mask=mask, per_page=per_page, page=page
    ).response().result
    all_contribs.extend(contribs)
    if not contribs or len(contribs) < per_page:
        break

print('found', len(all_contribs), 'materials')
all_contribs[-1]